In [137]:
import json
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from typing import *
from pprint import pprint

# Ergebnisse laden

In [138]:
def get_df_from_results(path_to_results:str) -> pd.DataFrame:
    with open(path_to_results, "r") as f:
        eval_results = f.readlines()
    eval_results = "\n".join(eval_results[17:])
    eval_results = json.loads(eval_results)
    eval_results = pd.DataFrame(eval_results).set_index("index")
    return eval_results

eval_results_path = r"./experiments/LLaMA2/lora/meld/True_two_unbalanced/preds_for_eval.text"
eval_results = get_df_from_results(eval_results_path)
eval_results.head()

,input,output,target
index,,,
0,Now you are expert of sentiment and emotional ...,neutral,neutral
1,Now you are expert of sentiment and emotional ...,neutral,neutral
2,Now you are expert of sentiment and emotional ...,angry,angry
3,Now you are expert of sentiment and emotional ...,neutral,angry
4,Now you are expert of sentiment and emotional ...,angry,surprise


# Untersuchen der Ergebnisse

In [139]:
def get_classification_report(eval_results:pd.DataFrame) -> pd.DataFrame:
    res = classification_report(eval_results["target"], eval_results["output"], zero_division=0.0, output_dict=True, digits=2)
    res = pd.DataFrame(res)
    res = res.drop(["accuracy", "macro avg", "weighted avg"], axis=1).swapaxes(0,1).reset_index()
    res.rename(columns={"index": "label"}, inplace=True)
    res = res.round(2)
    res["support"] = res["support"].astype(int)
    res.sort_values("f1-score", ascending=False, inplace=True)

    return res

get_classification_report(eval_results)

,label,precision,recall,f1-score,support
10,neutral,0.76,0.84,0.80,1256
9,joyful,0.68,0.53,0.60,402
13,surprise,0.57,0.62,0.59,281
0,angry,0.56,0.53,0.54,345
2,disgust,0.38,0.44,0.41,68
12,sad,0.48,0.34,0.40,208
6,fear,0.35,0.28,0.31,50
1,curious,0.00,0.00,0.00,0
3,embarrassed,0.00,0.00,0.00,0
4,excited,0.00,0.00,0.00,0


## Falsche Output-Labels
In der Tabelle sind zu viele Emotionen enthalten.
Dafür suche ich jetzt die Ursache

In [140]:
target_labels = eval_results["target"].unique()
predicted_labels = eval_results["output"].unique()

print(f"Gewünschte Labels: {', '.join(target_labels)}.")
print(f"Erhaltene Labels: {', '.join(predicted_labels)}.")

Gewünschte Labels: neutral, angry, surprise, sad, fear, joyful, disgust.
Erhaltene Labels: neutral, angry, surprise, joyful, fear, sad, disgust, hopeful, relieved, curious, embarrassed, excited, greedy, fascinated.


Das Modell scheint auch Emotionen zu benennen, die nicht gefordert sind. Welchen Anteil machen diese aus?

In [141]:
wrong_targets = eval_results[~eval_results["output"].isin(target_labels)]
print(f"Es gibt {len(wrong_targets)} falsche Labels")
print(f"Das entspricht {round((len(wrong_targets) / len(eval_results)) * 100, 2)}% des Datensatzes")

Es gibt 8 falsche Labels
Das entspricht 0.31% des Datensatzes


Diese Anzahl ist vernachlässigbar. Zur weiteren Analyse werden diese Zeilen ignoriert.

In [142]:
eval_results = eval_results[eval_results["output"].isin(target_labels)]

## Ergebnisse

In [143]:
from lets_plot import *
from lets_plot.mapping import as_discrete
LetsPlot.setup_html()

### Erfolg in Abhängigkeit der Anzahl Samples

In [144]:
cls_tabel = get_classification_report(eval_results)
display(cls_tabel.sort_values("f1-score", ascending=False))

ggplot(cls_tabel, aes(y="f1-score", x="support", label="label")) + \
    geom_point() + \
    geom_text(vjust="bottom", nudge_y=0.02) + \
    xlim(0, 1300) + \
    ylim(0, 1)

,label,precision,recall,f1-score,support
4,neutral,0.76,0.84,0.80,1253
3,joyful,0.68,0.54,0.60,398
6,surprise,0.57,0.62,0.59,281
0,angry,0.56,0.53,0.54,345
1,disgust,0.38,0.44,0.41,68
5,sad,0.48,0.34,0.40,208
2,fear,0.35,0.29,0.31,49


Im Plot lässt sich eine Klare Korrelation zwischen der Anzahl Beispielen und dem F1-Score festmachen. Eine neutale Stimmung ist im (Test-) Datensatz mehr als 3x so oft vertreten, wie die zweithäufigste Emotion (joyful). Aktuell gibt es keine Skallierung des Fehlers beim Training, hier könnte man nochmal rein gucken

### Confusion Matrix

By definition a confusion matrix $C$ is such that $C_{ij}$ is equal to the number of observations known to be in group $i$ and predicted to be in group $j$.
Thus in binary classification, the count of true negatives is $C_{0,0}$, false negatives is $C_{1,0}$, true positives is $C_{1,1}$ and false positives is $C_{0,1}$. [Ref](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)

Index: y_true  
Column: y_pred

In [145]:
import numpy as np

def print_confusion_matrix(results:pd.DataFrame, target_labels:Union[List[str],None] = None) -> None:
    target_labels = results["target"].unique() if target_labels is None else target_labels
    cm = confusion_matrix(results["target"], results["output"], labels = target_labels)
    cm_percent:np.ndarray = (np.sqrt(cm.astype('float')) / np.sqrt(cm.sum(axis=1))[:, np.newaxis])

    img_coordinates = np.arange(len(target_labels)) + 0.5
    x, y = np.meshgrid(img_coordinates, img_coordinates[::-1])
    pairs = np.column_stack((x.ravel(), y.ravel()))

    cm_labels = pd.DataFrame({"labels": cm.ravel(), "x": pairs[:,0], "y": pairs[:,1]})

    p = ggplot() + \
            geom_imshow(cm_percent, vmin=0, vmax=1, cmap="magma",show_legend=False) + \
            geom_text(aes(label = "labels", x = "x", y = "y"), data = cm_labels, size = 8, color="white") + \
            scale_x_discrete(labels=target_labels, breaks = img_coordinates, position="top") + \
            scale_y_discrete(labels=target_labels[::-1], breaks = img_coordinates) + \
            xlab("Predicted") + \
            ylab("True") + \
            theme(axis_line_x='blank', legend_title={"title": "Prozentualer Anteil pro Zeile"})
    
    display(p)

print_confusion_matrix(eval_results, target_labels)

#### Auffälligkeiten
- In den meisten Falschklassifizierungen wird "neutral" vorhergesagt
- "suprise" und "joyful" werden häufiger miteinander verwechselt
    - Ist mMn auch recht schwer auseinanderzuhalten bzw. eine Emotion, die sich oft auch überschneiden kann (z.B. freudiges überrascht werden)
- "angry" und "supprise" werden häufiger miteinander verwechselt
    - Hier könnten akkustische Feature hilfreich sein
- Die schlechteste Erkennung hat "fear". Sowofl der F1-Score, als auch die Anzahl Beispiele sind hier am geringsten

**Wo könnten akustische Feature helfen?**
- Ich denke besonders in der Abgrenzung zu "neutral", da eine neutrale Tonlage besonders von *stärkeren* Emotionen unterscheidbar ist. Bei "angry" oder "joyful" stelleich mir vor, dass sich die Tonlage stark verändert und wahrscheinlich auch das Sprechtempo.

## Leistung in Abhängigkeit vom Text
Welchen Einfluss haben Textlänge und Fortschritt, Anzahl der Sprecher ... im Gespräch auf die Leistung?

In [234]:
import re
from nltk.tokenize import word_tokenize
def get_dialog(prompt:str) -> Tuple[List[str], Set[str]]:
    base_prompt = "Now you are expert of sentiment and emotional analysis. The following conversation noted between '### ###' involves several speakers."
    prompt = prompt.replace(base_prompt, "")
    dialog = re.findall(r"###(.*)###", prompt)[0]
    dialog_text = re.findall(r"Speaker_\d+:\"(.*?)(?=\"\t|\"\s?$)", dialog, re.MULTILINE)
    involved_speakers = set(re.findall(r"(Speaker_\d+)", dialog))
    return dialog_text, involved_speakers

def get_dialog_features(results:pd.DataFrame) -> pd.DataFrame:
    feature = results.copy()
    feature["num_speakers"] = feature["input"].apply(lambda x: len(get_dialog(x)[1]))
    feature["dialog_length"] = feature["input"].apply(lambda x: len(get_dialog(x)[0]))
    feature["target_utterence"] = feature["input"].apply(lambda x: get_dialog(x)[0][-1])
    feature["target_utterence_length"] = feature["target_utterence"].apply(lambda x: len(x.split()))
    feature["correct"] = feature["output"] == feature["target"]
    feature["correct"] = feature["correct"].astype(str)
    return feature

eval_results = get_dialog_features(eval_results)
display(eval_results.sort_values("target_utterence_length").head())
display(eval_results.sort_values("target_utterence_length").head().iloc[0,0])


,input,output,target,num_speakers,dialog_length,target_utterence,target_utterence_length,correct
index,,,,,,,,
629,Now you are expert of sentiment and emotional ...,neutral,joyful,2,14,What?,1,False
1850,Now you are expert of sentiment and emotional ...,angry,disgust,3,6,Arghh!!,1,False
1233,Now you are expert of sentiment and emotional ...,joyful,neutral,3,3,Hi!,1,False
1235,Now you are expert of sentiment and emotional ...,neutral,neutral,3,5,Bye!,1,True
2085,Now you are expert of sentiment and emotional ...,joyful,joyful,1,1,Hi!,1,True


'Now you are expert of sentiment and emotional analysis. The following conversation noted between \'### ###\' involves several speakers. ### \t Speaker_0:"Come on Joey!!!"\t Speaker_1:"Rach, I told you everything I knew last night!"\t Speaker_1:"Look, it\'s not that big of a deal, so Monica and Chandler are doing it."\t Speaker_0:"I can\'t believe you would say that!"\t Speaker_1:"Sorry. Monica and Chandler are making love."\t Speaker_0:"No! I mean come on! This is a"\t Speaker_1:"I don\'t know."\t Speaker_0:"Is he romantic with her?"\t Speaker_1:"I don\'t know."\t Speaker_0:"Are they in love?"\t Speaker_1:"I don\'t know."\t Speaker_0:"You don\'t know anything."\t Speaker_1:"Ohh, I know one thing!"\t Speaker_0:"What?" ### Please select the emotional label of < Speaker_0:"What?"> from <neutral, surprise, fear, sad, joyful, disgust, angry>:'

### Einfluss der Sprechermenge

In [245]:
def normalized_results(results:pd.DataFrame, column:str) -> pd.DataFrame:
    num_frequencys = np.bincount(results[column])
    grouped_freqs = results.value_counts(["correct", column], normalize=False).unstack().swapaxes(0,1).reset_index()
    num_frequencys = num_frequencys[grouped_freqs[column]]
    grouped_freqs[["True", "False"]] = grouped_freqs[["True", "False"]].apply(lambda x: x / num_frequencys, axis=0)
    num_proportion = grouped_freqs.melt(id_vars=[column], value_vars=["True", "False"])
    return num_proportion

def normalized_bar_plot(results:pd.DataFrame, column:str, title:str = None, xlabel:str = None, ylabel:str = "Proportion") -> None:
     num_proportion = normalized_results(results, column)
     p = ggplot(num_proportion, aes(x=as_discrete(column), y="value", fill="correct")) + \
            geom_bar(position="dodge", stat="identity") + \
            ylab(ylabel) + \
            xlab(xlabel) + \
            ggtitle(title)
     display(p)

normalized_bar_plot(eval_results, "num_speakers", "Anteile (in-)korrekter Vorhersagen je Anzahl der Sprecher", "Anzahl der Sprecher")

Die Vorhersagequalität scheint unabhängig von der Anzahl am Gespräch beteiligter Personen zu sein. <br>
Dennoch ist ein leichter abfallender Trend zu erkennen, wo mit der Anzahl Sprecher die Qualität der Vorhersage abnimmt.<br>
Zu berücksichtigen ist aber auch, dass die Anzahl der Beispiele für viele Sprecher weit aus kleiner ist, als für wenige.

In [163]:
ggplot(eval_results, aes(x="num_speakers")) + geom_bar()

### Einfluss der Dialoglänge
*Note: Die Dialoghistorie wurde auf 20 limitiert*

In [246]:
normalized_bar_plot(eval_results, "dialog_length", "Anteile (in-)korrekter Vorhersagen pro Dialoglänge", "Dialoglänge")

Es ist kein Trend zu erkennen. Da die Peak-Performance bei 3 Schritten (Historie von 2) liegt, könnte man ausprobieren, wie sich ein kleineres Fenster auf die Qualität auswirkt. Allerdings kann dies auch reiner Zufall sein.

In [171]:
ggplot(eval_results, aes(x="dialog_length")) + geom_bar()

### Einfluss der zu klassifizierenden Textlänge 

In [354]:
normalized_tul = normalized_results(eval_results, "target_utterence_length")
normalized_tul = normalized_tul[normalized_tul["correct"] == "True"]
normalized_tul.fillna(0, inplace=True)
bin_width = 4
tul_dist = np.bincount(eval_results["target_utterence_length"])
# tul_dist["target_utterence_length"] -= bin_width/2
display(tul_dist)
tul_dist["count"] /= len(eval_results)
ggplot(mapping = aes(x="target_utterence_length")) + \
    geom_line(aes(y="value"), data = normalized_tul, color = "#118ed8") + \
    geom_freqpoly(aes(y="count"), data = tul_dist)

array([  0, 310, 192, 213, 203, 207, 152, 152, 147, 122, 131, 115,  76,
        74,  73,  66,  72,  36,  55,  41,  24,  27,  29,  23,  18,   7,
         7,   9,   6,   4,   2,   1,   1,   1,   3,   1,   0,   0,   0,
         0,   0,   1,   0,   0,   0,   1])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

input                      192
output                     192
target                     192
num_speakers               192
dialog_length              192
target_utterence           192
target_utterence_length    192
correct                    192
dtype: int64

In [263]:
ggplot(eval_results, aes(x="target_utterence_length")) + geom_bar()

# Klassengewichte

Die Confusionmatrix legt nah, dass ein Problem bei der Klassifizierung die Klassenbalance ist. Besonders "fear" wird häufiger als "neutral" klassifiziert als "fear" selbst. <br>
Daher nutze ich in einem weiteren Versuch Klassengewichte, um den Fehlereinfluss der Klassen zu verändern

Berechnung der Klassengewichte mittels `class_weights = n_samples / (n_classes * np.bincount(y))`. class_weights ist ein Vektor, der die Gewichte pro Klasse enthält. <br>
Da ein so starkes Ungleichgewicht besteht, müssen die Gewichte etwas abgeschwächt werden, um keinen übermäßigen Fehler zu erzeugen. Dafür wird folgende Funktion verwendet:
$$
f(x) = 2 - \frac{1}{0.5 + 0.5\cdot \exp(-\alpha x)}
$$
Für $\alpha = 1$ ergibt sich die Kurve im folgenden Plot

In [36]:
def inv_sigmoid(x, alpha = 1):
    return 2 - (1 / (0.5 + 0.5*np.exp(-alpha*x)))

x = np.linspace(0, 10, 100)
y = inv_sigmoid(x, 1)
ggplot({'x': x, 'y': y}, aes(x='x', y='y')) + geom_line()

In [37]:
with open(r"./processed_data/meld/predict/window/train.json", "r") as f:
    datarows = f.readlines()
datarows = [json.loads(d) for d in datarows]
train_labels = [d["target"] for d in datarows]

In [38]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced', classes=target_labels, y=train_labels)
label_weight_mapping = {l:w for l, w in zip(target_labels, class_weights*0.5)}
label_weight_mapping = pd.DataFrame(zip(target_labels, class_weights), columns=["label", "weight"])
label_weight_mapping["new_weight"] = label_weight_mapping["weight"] * inv_sigmoid(label_weight_mapping["weight"], alpha=0.2)
ggplot(label_weight_mapping, aes(x="label", y="weight")) + geom_bar(stat="identity") + geom_bar(aes(y="new_weight"), stat="identity", fill="red")


## Ergebnis

In [40]:
balanced_results = get_df_from_results(r"./experiments/LLaMA2/lora/meld/True_two_balanced/preds_for_eval.text")
balanced_results = balanced_results[balanced_results["output"].isin(balanced_results["target"].unique())]
display(cls_tabel)
display(get_classification_report(balanced_results))
print_confusion_matrix(balanced_results)

,label,precision,recall,f1-score,support
4,neutral,0.76,0.84,0.80,1253
3,joyful,0.68,0.54,0.60,398
6,surprise,0.57,0.62,0.59,281
0,angry,0.56,0.53,0.54,345
1,disgust,0.38,0.44,0.41,68
5,sad,0.48,0.34,0.40,208
2,fear,0.35,0.29,0.31,49


,label,precision,recall,f1-score,support
4,neutral,0.79,0.80,0.79,1253
6,surprise,0.53,0.68,0.59,281
3,joyful,0.67,0.51,0.58,402
0,angry,0.50,0.51,0.50,345
1,disgust,0.40,0.44,0.42,68
5,sad,0.46,0.39,0.42,208
2,fear,0.24,0.35,0.28,49


**Gewichte haben leider in allen, bis auf zwei Kategorien, schlechtere Ergebnisse erzielt**